In [23]:
import numpy as np
import numba

In [29]:
@numba.njit
def make_key(i1, i2):
    if i1 < i2:
        return (i1, i2)
    return (i2, i1)

@numba.jit
def flip_one_edge(pts, tris, edge2tris, pnt2tris, edge):
    res = set()
    _tris = edge2tris.get(edge, [])

    iTri1, iTri2 = _tris
    tri1 = tris[iTri1]
    tri2 = tris[iTri2]

    # find the opposite vertices, not part of the edge
    iOpposite1 = -1
    iOpposite2 = -1
    for i in range(3):
        if not tri1[i] in edge:
            iOpposite1 = tri1[i]
        if not tri2[i] in edge:
            iOpposite2 = tri2[i]

    newTri1 = [iOpposite1, edge[0], iOpposite2]  # triangle a
    newTri2 = [iOpposite1, iOpposite2, edge[1]]  # triangle b
    tris[iTri1] = newTri1
    tris[iTri2] = newTri2
    del edge2tris[edge]

    e = make_key(iOpposite1, iOpposite2)
    edge2tris[e] = [iTri1, iTri2]
    pnt2tris[e[0]].update({iTri1, iTri2})
    pnt2tris[e[1]].update({iTri1, iTri2})

    e = make_key(iOpposite1, edge[1])
    v = edge2tris[e]
    for i in range(len(v)):
        if v[i] == iTri1:
            v[i] = iTri2
    res.add(e)

    e = make_key(iOpposite2, edge[0])
    v = edge2tris[e]
    for i in range(len(v)):
        if v[i] == iTri2:
            v[i] = iTri1
    res.add(e)
    
    for i in newTri1:
        if i in edge:
            tr = list(pnt2tris[i])
            for j in range(len(tr)):
                if tr[j] == iTri2:
                    tr[j] = iTri1
            pnt2tris[i] = set(tr)

    for i in newTri2:
        if i in edge:
            tr = list(pnt2tris[i])
            for j in range(len(tr)):
                if tr[j] == iTri1:
                    tr[j] = iTri2
            pnt2tris[i] = set(tr)
    
    res.add(make_key(iOpposite1, edge[0]))
    res.add(make_key(iOpposite2, edge[1]))

    return res

In [30]:
pts = np.array([
    [0.4, 0.7],
    [0.6, 0.7],
    [0.2, 0.5],
    [0.8, 0.5],
    [0. , 0. ],
    [1. , 0. ]])
tris = [[0, 2, 1], [1, 2, 3]]
edge2tris = {(0, 2): [0], (1, 2): [0, 1], (0, 1): [0], (2, 3): [1], (1, 3): [1]}
pnt2tris = {0: {0}, 2: {0, 1}, 1: {0, 1}, 3: {1}}
edge = (1, 2)

flip_one_edge(pts, tris, edge2tris, pnt2tris, edge)

{(0, 1), (0, 2), (1, 3), (2, 3)}